In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction to Generative AI functions in BigQuery

## Overview

This tutorial will guide you through powerful generative AI capabilities available in BigQuery. You'll get hands-on experience using part of the suite of generative **`AI.*` functions** that integrate directly with powerful models like Gemini. This allows you to perform sophisticated AI-driven analysis on your data right within your familiar SQL environment.

While the examples in this notebook will focus on **text inputs** to the generative AI models, many of these capabilities extend to **multi-modal analysis**.

### Objectives

We'll cover how to:

* **Prompt models to generate text and structured data** with `AI.GENERATE` and `AI.GENERATE_TABLE`


---

## 1. Before you begin

### 1.1. Set your project ID

In [1]:
PROJECT_ID_LIST=!gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID=PROJECT_ID_LIST[0]
LOCATION="us-central1"
DATA_DS="rscw_oltp_stg_ds"
MULTIMODAL_STAGE_DS="rscw_multimodal_ai_stg_ds"

### 1.2. Create a BigQuery Dataset

Running the following query creates a BigQuery dataset called `rscw_multimodal_ai_stg_ds` to house tables created in this tutorial:

In [2]:
%%bigquery --project {PROJECT_ID}

CREATE SCHEMA `rscw_multimodal_ai_stg_ds` OPTIONS (location = 'us-central1');

Executing query with job ID: fc91b188-6b06-473b-8149-703a1859cec7
Query executing: 0.20s


ERROR:
 409 GET https://bigquery.googleapis.com/bigquery/v2/projects/data-insights-quickstart/queries/fc91b188-6b06-473b-8149-703a1859cec7?maxResults=0&location=us-central1&prettyPrint=false: Already Exists: Dataset data-insights-quickstart:rscw_multimodal_ai_stg_ds

Location: us-central1
Job ID: fc91b188-6b06-473b-8149-703a1859cec7



### 1.3. Create BigQuery Cloud resource connection

You will need to create a [Cloud resource connection](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) to enable BigQuery to interact with Vertex AI services.

In [3]:
!bq mk --connection --location=us-central1 \
    --connection_type=CLOUD_RESOURCE rscw_cloud_resource_connection_usc

BigQuery error in mk operation: Already Exists: Connection
projects/606804615020/locations/us-
central1/connections/rscw_cloud_resource_connection_usc


### 1.4. Set permissions for Service Account

The resource connection service account requires certain project-level permissions to interact with Vertex AI.

In [4]:
SERVICE_ACCT = !bq show --format=prettyjson --connection us-central1.rscw_cloud_resource_connection_usc | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print(SERVICE_ACCT_EMAIL)

bqcx-606804615020-e7x5@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [5]:
import time

!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/aiplatform.user'

# wait 60 seconds, give IAM updates time to propagate, otherwise, following cells will fail
time.sleep(60)

Updated IAM policy for project [data-insights-quickstart].
Updated IAM policy for project [data-insights-quickstart].


---

## 2. Generate text and structured data with `AI.GENERATE` and `AI.GENERATE_TABLE`

These functions allow you to leverage the power of large language models (LLMs) directly within BigQuery to generate new text content, including creating content with a specified schema. Both functions work by sending requests to your choice of a [generally available](https://cloud.google.com/vertex-ai/generative-ai/docs/models#generally_available_models) or [preview](https://cloud.google.com/vertex-ai/generative-ai/docs/models#preview_models) Gemini model, and then returning that model's response.

### 2.1. Using `AI.GENERATE`: Generate product descriptions

Let's use the [`AI.GENERATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate) function to generate product descriptions for the products in the `rscw_oltp_stg_ds.product_master` table and persist to `rscw_multimodal_ai_stg_ds.product_master_augmented` table. We will also generate some fake reviews of the products for subsequent sections.

In [6]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_master_augmented` AS
SELECT
  product_id,
  product_nm,
  AI.GENERATE(('Generate a product description for this product: ', product_nm),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    endpoint => 'gemini-2.5-flash').result AS product_description,
  AI.GENERATE(('Generate a 70-30 balance of positive and negative reviews for the product, generate 10 reviews and as json called reviews but without any backticks and without the word json in the review payload. Also, the review should have just a rating and a comment: ', product_nm),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    endpoint => 'gemini-2.5-flash').result AS product_reviews,
  '' AS product_image_gcs_uri,
  '' AS product_user_manual_gcs_uri
FROM
  `rscw_oltp_stg_ds.product_master`
WHERE
  category_nm = 'home_appliances'
AND
  product_id in ('c2f2d10f8b047f5cffef731479a34432' ,'e9aa3fd3f316b44c85f671552a04fa4a' ,'d418ddef3e7017381084956220b00b9d' ,'e403a54af54730eb4f3e8a346d943579' ,'ba9a847a2c023953743ba9e5ef6c88c6' ,'832137af5164ffb89cf407d686aa2d0d' ,'900d210e9ea6ee40dd0290df76bde9fc' ,'520ac02e0e8cdf179eed22f204e3f26d' ,'aa15d9042b3749f119a734d7df2b68bc' ,'ccb125d39320e9d900107ce4109589dd')
LIMIT 10;


SELECT product_id, product_nm,product_description
FROM rscw_multimodal_ai_stg_ds.product_master_augmented LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_nm,product_description
0,e9aa3fd3f316b44c85f671552a04fa4a,QuickPress Mini Iron,## QuickPress Mini Iron: Your Secret Weapon Ag...
1,ba9a847a2c023953743ba9e5ef6c88c6,Portable Fabric Steamer,"## Effortless Style, Anywhere You Go: Unleash ..."


### 2.2. Using `AI.GENERATE`: Extract keywords from reviews

Let's use the [`AI.GENERATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate) function to extract keywords from the product reviews in the `rscw_multimodal_ai_stg_ds.product_master_augmented` table.

To do this, lets first flatten the product_reviews json for each product into a separate table `rscw_multimodal_ai_stg_ds.product_reviews`

In [7]:
%%bigquery --project {PROJECT_ID}

-- 1) Unnest the ratings JSON into rows

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_reviews` AS
SELECT t.product_id, t.product_nm, JSON_EXTRACT_SCALAR(review, '$.rating') AS review_rating,JSON_EXTRACT_SCALAR(review, '$.comment') AS review_comment
FROM
    rscw_multimodal_ai_stg_ds.product_master_augmented  AS t,
    UNNEST(JSON_EXTRACT_ARRAY(t.product_reviews, '$.reviews')) AS review;

SELECT product_id, product_nm,review_rating, review_comment
FROM rscw_multimodal_ai_stg_ds.product_reviews LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_nm,review_rating,review_comment
0,aa15d9042b3749f119a734d7df2b68bc,Electric Hand Mixer,5,This hand mixer exceeded my expectations. The ...
1,c2f2d10f8b047f5cffef731479a34432,InstaSteam Garment Steamer,4,I'm impressed with how effective the InstaStea...


In [8]:
%%bigquery --project {PROJECT_ID}

--2a) Extract keywords

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords from the text, dont preface it with any sentence, just return the keywords extracted ', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    endpoint => 'gemini-2.5-flash').result AS keywords
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,keywords
FROM rscw_multimodal_ai_stg_ds.product_review_keywords LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords
0,520ac02e0e8cdf179eed22f204e3f26d,Excellent quality and thickness. It stays perf...,"Excellent quality, Thickness, Stays in place, ..."
1,520ac02e0e8cdf179eed22f204e3f26d,A total game-changer for my kitchen sink area....,"game-changer, kitchen sink area, standing, pai..."


You can see that BigQuery has prompted the specified model for each row of the data and returned the response.

#### Using arguments to adjust model configuration
In addition to the `prompt`, `connection_id`, and `endpoint`, there are additional arguments available in `AI.GENERATE` at your disposal for customizing your generation request.

##### The `model_params` argument
There are several parameters you can specify within the `model_params` argument. You can read more about which specific parameters are available for `AI.GENERATE` in the [documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate#arguments). The next query adds custom `temperature` and `maxOutputTokens` parameters.

* The `temperature` parameter controls the randomness of the response, where a lower value makes the output more predictable and focused, while a higher value encourages more creative and diverse results.

* The `maxOutputTokens` parameter sets the maximum length of the model's response by limiting the total number of tokens (pieces of words) it can generate.

You can read more about the various parameters [here](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#generationconfig), and the parameters' default values are documented in the model card in the [model documentation for Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/models).

In [9]:
%%bigquery --project {PROJECT_ID}

--2b) Extract keywords and specify temperature and max output tokens

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords from the text, dont preface it with any sentence, just return the keywords extracted ', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    model_params => JSON '{"generationConfig":{"temperature": 0.5, "maxOutputTokens": 1000}}',
    endpoint => 'gemini-2.5-flash').result AS keywords
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,keywords
FROM rscw_multimodal_ai_stg_ds.product_review_keywords LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords
0,520ac02e0e8cdf179eed22f204e3f26d,A total game-changer for my kitchen sink area....,game-changer\nkitchen sink area\npainful\nbreeze
1,520ac02e0e8cdf179eed22f204e3f26d,Good cushioning and surprisingly easy to clean...,"Good cushioning, easy to clean, comfort layer,..."


Why are some of the responses empty? By setting the `maxOutputTokens` value to 1,000 (much lower than the default 65,535 which was used in the first queries), you are seeing this limit in action. Gemini 2.5 Flash has [thinking capabilities](https://cloud.google.com/vertex-ai/generative-ai/docs/thinking), which means the model goes through a "thinking process" before determining and returning its response, and this thinking process consumes output tokens. In this case, the model is hitting its `maxOutputTokens` value before it moves from its thinking process to returning the actual response.

You can resolve this issue by increasing the `maxOutputTokens` value and/or setting a `thinking_budget` to limit the amount of tokens that can be consumed by the thinking process.

###### Setting a `thinking_budget`

You can set a [`thinking_budget`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate#thinking-budget) to manage how many output tokens are allocated to the thinking phase of the model's response development.

The next query shows how to set the `thinking_budget` as an additional parameter in the `model_params` argument.

In [10]:
%%bigquery --project {PROJECT_ID}

--2c) Extract keywords, specify temperature, max output tokens, and a specific thinking budget

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords from the text, dont preface it with any sentence, just return the keywords extracted ', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    model_params => JSON '{"generationConfig":{"temperature": 0.5, "maxOutputTokens": 1000, "thinking_config": {"thinking_budget": 500}}}',
    endpoint => 'gemini-2.5-flash').result AS keywords
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,keywords
FROM rscw_multimodal_ai_stg_ds.product_review_keywords LIMIT 2;


Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords
0,520ac02e0e8cdf179eed22f204e3f26d,Not as thick as I expected from the pictures. ...,"thick, expected, pictures, discomfort, standin..."
1,520ac02e0e8cdf179eed22f204e3f26d,Durable material and it doesn't move around on...,"Durable material, doesn't move, tiled floor, k..."


##### The `output_schema` argument

As you can see in the last query, the model's responses were not provided in a standard structure. You can use the **`output_schema`** argument to define the format of the responses.

First, you'll specify the output as a string of key words by adding `output_schema => 'keywords ARRAY<STRING>'` to the `AI.GENERATE` request.

In [11]:
%%bigquery --project {PROJECT_ID}


--2d) Extract keywords, specify temperature, max output tokens, a specific thinking budget & with output schema

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords from the text, dont preface it with any sentence, just return the keywords extracted ', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    model_params => JSON '{"generationConfig":{"temperature": 0.5, "maxOutputTokens": 1000, "thinking_config": {"thinking_budget": 500}}}',
    endpoint => 'gemini-2.5-flash',
    output_schema => 'keywords ARRAY<STRING>'
    ).keywords AS keywords
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,keywords
FROM rscw_multimodal_ai_stg_ds.product_review_keywords LIMIT 2;


Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords
0,520ac02e0e8cdf179eed22f204e3f26d,The perfect size for my needs and incredibly s...,"[perfect size, supportive, mat, recommend, on ..."
1,520ac02e0e8cdf179eed22f204e3f26d,Durable material and it doesn't move around on...,"[Durable material, tiled floor, knees, ankles,..."


Now the responses are now arrays of keywords - a more useful format for this use case!

Next, you'll adjust the prompt to the model to extract the sentiment as well, and adjust the `output_schema` accordingly.

In [12]:
%%bigquery --project {PROJECT_ID}

--2d) Extract keywords, specify temperature, max output tokens, and a specific thinking budget
--    and with sentiment as well

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.product_review_keywords_sentiment` AS
SELECT
  product_id,
  review_comment,
  AI.GENERATE(('Extract the keywords and sentiment (Positive, Negative, or Neutral) from the review_comment; Return both keywords and sentiment', review_comment),
    connection_id => 'us-central1.rscw_cloud_resource_connection_usc',
    model_params => JSON '{"generationConfig":{"temperature": 0.5, "maxOutputTokens": 1000, "thinking_config": {"thinking_budget": 500}}}',
    endpoint => 'gemini-2.5-flash',
    output_schema => 'keywords ARRAY<STRING>, sentiment STRING').full_response
FROM
  `rscw_multimodal_ai_stg_ds.product_reviews`
;

SELECT product_id,review_comment,full_response
FROM rscw_multimodal_ai_stg_ds.product_review_keywords_sentiment LIMIT 2;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,full_response
0,900d210e9ea6ee40dd0290df76bde9fc,"The handle isn't as ergonomic as I hoped, star...","{""candidates"":[{""avg_logprobs"":-0.465537500381..."
1,900d210e9ea6ee40dd0290df76bde9fc,"While it whisks fine, I expected it to be 'qui...","{""candidates"":[{""avg_logprobs"":-1.658523393713..."


This time, the response is in a JSON format. Why?

When using `output_schema` with [`AI.GENERATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate), you can specify in the SQL which field you want to return. In this case the SQL specifies the `.full_response` which will provide a JSON object containing the various text response elements. You may alternatively specify to return any one of the schema columns you defined, such as `.keywords` (as was used in the first query of this section) or `.sentiment`. However, you cannot specify to return multiple columns.

 You could use SQL to parse the JSON provided in the `.full_response`,  but for ease of use, let's look at the [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) function.

### 2.3. Using `AI.GENERATE_TABLE`: Extract keywords and sentiment from product reviews

The [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) function provides expanded functionality for turning unstructured text into a structured table. Here, you'll perform the same analysis to extract keywords and overall sentiment from the `rscw_multimodal_ai_stg_ds.product_review` table and return the response as multiple columns in one query.

#### Note:

The syntax for using [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) differs from [`AI.GENERATE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-ai-generate), and it requires that you first create a remote model in BigQuery that connects to the one of the [generally available](https://cloud.google.com/vertex-ai/generative-ai/docs/models#generally_available_models) or [preview](https://cloud.google.com/vertex-ai/generative-ai/docs/models#preview_models) Gemini models.

We will house the remote model in the BigQuery dataset  `rscw_multimodal_ai_stg_ds`

#### 2.3.1. Create a remote model for Gemini 2.5 Flash

Now you'll create the remote model, which is simply a pointer to the model endpoint. In this example, you'll continue to use the `gemini-2.5-flash` model endpoint.

Once you create this remote model, it is saved in your `rscw_multimodal_ai_stg_ds` dataset so that you can use it in any future analysis.

In [13]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE MODEL
  `rscw_multimodal_ai_stg_ds.gemini_2_5_flash`
REMOTE WITH
    CONNECTION `us-central1.rscw_cloud_resource_connection_usc`
    OPTIONS (ENDPOINT = 'gemini-2.5-flash');


Query is running:   0%|          |

""


#### 2.3.2. Run analysis using the remote model

Now that the remote model has been created, you can prompt it with [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) to analyze data and specify the output schema with multiple columns returned.

In [14]:
%%bigquery --project {PROJECT_ID}


SELECT
  product_id,
  review_comment,
  keywords,
  sentiment
FROM
  AI.GENERATE_TABLE(
    MODEL `rscw_multimodal_ai_stg_ds.gemini_2_5_flash`,
    (
      SELECT
        product_id,
        review_comment,
        ('Extract the keywords and sentiment (Positive, Negative, or Neutral) from the following review:',review_comment) AS prompt,
      FROM
        `rscw_multimodal_ai_stg_ds.product_reviews`
    ),
    STRUCT(
      'keywords ARRAY<STRING>, sentiment STRING' AS output_schema
    )
  );

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords,sentiment
0,e403a54af54730eb4f3e8a346d943579,So incredibly convenient! It takes up hardly a...,"[convenient, hardly any space, blends beautifu...",Positive
1,e403a54af54730eb4f3e8a346d943579,Super compact and great for travel. It blends ...,"[compact, travel, blending performance, batter...",Positive
2,e9aa3fd3f316b44c85f671552a04fa4a,This mini iron is a game-changer for my crafti...,"[mini iron, crafting projects, precision tip, ...",Positive
3,900d210e9ea6ee40dd0290df76bde9fc,"A good solid whisk, much better than my old fl...","[good solid whisk, flimsy one, baking, easier]",Positive
4,832137af5164ffb89cf407d686aa2d0d,"I had high hopes, but this sealer stopped work...","[sealer, stopped working, vacuum, frustrating]",Negative
...,...,...,...,...
85,832137af5164ffb89cf407d686aa2d0d,The Handy-Helper sealer performs exactly as ad...,"[Handy-Helper sealer, intuitive to operate, ef...",Positive
86,900d210e9ea6ee40dd0290df76bde9fc,Excellent for whipping eggs and light batters....,"[whipping eggs, light batters, saves time, sav...",Positive
87,ba9a847a2c023953743ba9e5ef6c88c6,"I travel a lot for work, and this steamer has ...","[travel, work, steamer, essential item, suitca...",Positive
88,e403a54af54730eb4f3e8a346d943579,Perfect for single servings! I've used it dail...,"[single servings, blends smoothly, no clumps, ...",Positive


#### 2.3.3. Saving results in a BigQuery table
Wrapping any `SELECT` statement with the [`CREATE TABLE` statement](https://cloud.google.com/bigquery/docs/reference/standard-sql/data-definition-language#create_table_statement) allows you to create a permanent table from your query results. You can run the next two queries to see that in action.

In [15]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `rscw_multimodal_ai_stg_ds.products_reviews_keywords_sentiment` AS (
  SELECT
    product_id,
    review_comment,
    keywords,
    sentiment
  FROM
    AI.GENERATE_TABLE(
      MODEL `rscw_multimodal_ai_stg_ds.gemini_2_5_flash`,
      (
        SELECT
          product_id,
          review_comment,
          ('Extract the keywords and sentiment (Positive, Negative, or Neutral) from the following review:',review_comment) AS prompt,
        FROM
          `rscw_multimodal_ai_stg_ds.product_reviews`
      ),
      STRUCT(
        'keywords ARRAY<STRING>, sentiment STRING' AS output_schema)));

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project {PROJECT_ID}

SELECT *
FROM `rscw_multimodal_ai_stg_ds.products_reviews_keywords_sentiment` LIMIT 10

#### 2.3.4. Using arguments to adjust model configuration
Similar to `AI.GENERATE`, [`AI.GENERATE_TABLE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table) also allows you to customize the parameters for your generation request. The placement of these parameters is within the `STRUCT` value. The [documentation page](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-table#:~:text=dataset.table%60%20%7C%20(query_statement)%20%7D%2C-,STRUCT(,-output_schema%20AS%20output_schema) contains the available fields you can define within the `STRUCT`.

Next you'll take the same query and now define values for `temperature` and `max_output_tokens`.

In [16]:
%%bigquery --project {PROJECT_ID}

SELECT
    product_id,
    review_comment,
    keywords,
    sentiment
  FROM
    AI.GENERATE_TABLE(
      MODEL `rscw_multimodal_ai_stg_ds.gemini_2_5_flash`,
      (
        SELECT
          product_id,
          review_comment,
          ('Extract the keywords and sentiment (Positive, Negative, or Neutral) from the following review:',review_comment) AS prompt,
        FROM
          `rscw_multimodal_ai_stg_ds.product_reviews`
      ),
      STRUCT(
        'keywords ARRAY<STRING>, sentiment STRING' AS Output_schema,
      0.5 AS temperature,
      1000 AS max_output_tokens));

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,review_comment,keywords,sentiment
0,832137af5164ffb89cf407d686aa2d0d,"I had high hopes, but this sealer stopped work...","[sealer, stopped working, vacuum, frustrating]",Negative
1,c2f2d10f8b047f5cffef731479a34432,This steamer heats up incredibly fast! Perfect...,"[steamer, heats up fast, quick touch-up, shirt...",Positive
2,e403a54af54730eb4f3e8a346d943579,Good value for the money. It's reliable for da...,"[Good value for the money, reliable for daily ...",Positive
3,ba9a847a2c023953743ba9e5ef6c88c6,Disappointed with this product. It leaks hot w...,"[disappointed, product, leaks, hot water, mess...",Negative
4,e403a54af54730eb4f3e8a346d943579,So incredibly convenient! It takes up hardly a...,"[convenient, blends beautifully, lifesaver, bu...",Positive
...,...,...,...,...
85,832137af5164ffb89cf407d686aa2d0d,The Handy-Helper sealer performs exactly as ad...,"[Handy-Helper sealer, performs exactly as adve...",Positive
86,900d210e9ea6ee40dd0290df76bde9fc,Excellent for whipping eggs and light batters....,"[whipping eggs, light batters, saves time, sav...",Positive
87,ba9a847a2c023953743ba9e5ef6c88c6,"I travel a lot for work, and this steamer has ...","[travel, work, steamer, essential item, suitca...",Positive
88,e403a54af54730eb4f3e8a346d943579,Perfect for single servings! I've used it dail...,"[single servings, daily use, blends smoothly, ...",Positive


### --

### This concludes the learning module. Proceed to the user manual.